In [1]:
import polars as pl

DATA_GLOB = "data/subset_part_*.parquet"   


In [2]:
lf = pl.scan_parquet(DATA_GLOB)

print("Schema:")
print(lf.schema)

print("Row count:")
print(lf.select(pl.len().alias("n_rows")).collect())


Schema:
Schema({'reviewerID': String, 'unixReviewTime': Int64, 'year': Int64, 'overall': Float64, 'asin': String, 'verified': Boolean, 'summary': String, 'reviewText': String})
Row count:
shape: (1, 1)
┌────────┐
│ n_rows │
│ ---    │
│ u32    │
╞════════╡
│ 200000 │
└────────┘


/var/folders/jc/m5_dzhb94fsdm0nt78jxzpx00000gn/T/ipykernel_11573/2055788536.py:4: PerformanceWarning: Resolving the schema of a LazyFrame is a potentially expensive operation. Use `LazyFrame.collect_schema()` to get the schema without this warning.
  print(lf.schema)


In [3]:
lf = lf.with_columns(
    pl.from_epoch("unixReviewTime", time_unit="s").alias("review_dt")
)

In [4]:
overview = lf.select([
    pl.len().alias("n_reviews"),
    pl.col("reviewerID").n_unique().alias("n_users"),
    pl.col("asin").n_unique().alias("n_products"),
    pl.min("year").alias("min_year"),
    pl.max("year").alias("max_year"),
]).collect()

print(overview)


shape: (1, 5)
┌───────────┬─────────┬────────────┬──────────┬──────────┐
│ n_reviews ┆ n_users ┆ n_products ┆ min_year ┆ max_year │
│ ---       ┆ ---     ┆ ---        ┆ ---      ┆ ---      │
│ u32       ┆ u32     ┆ u32        ┆ i64      ┆ i64      │
╞═══════════╪═════════╪════════════╪══════════╪══════════╡
│ 200000    ┆ 125233  ┆ 94636      ┆ 2018     ┆ 2018     │
└───────────┴─────────┴────────────┴──────────┴──────────┘


In [5]:
missing = lf.select([
    pl.all().null_count()
]).collect()
print(missing)


shape: (1, 9)
┌────────────┬────────────────┬──────┬─────────┬───┬──────────┬─────────┬────────────┬───────────┐
│ reviewerID ┆ unixReviewTime ┆ year ┆ overall ┆ … ┆ verified ┆ summary ┆ reviewText ┆ review_dt │
│ ---        ┆ ---            ┆ ---  ┆ ---     ┆   ┆ ---      ┆ ---     ┆ ---        ┆ ---       │
│ u32        ┆ u32            ┆ u32  ┆ u32     ┆   ┆ u32      ┆ u32     ┆ u32        ┆ u32       │
╞════════════╪════════════════╪══════╪═════════╪═══╪══════════╪═════════╪════════════╪═══════════╡
│ 0          ┆ 0              ┆ 0    ┆ 0       ┆ … ┆ 0        ┆ 106     ┆ 284        ┆ 0         │
└────────────┴────────────────┴──────┴─────────┴───┴──────────┴─────────┴────────────┴───────────┘


In [6]:
print(lf.group_by("overall").agg(pl.len().alias("n")).sort("overall").collect())
print(lf.group_by("year").agg(pl.len().alias("n")).sort("year").collect())


shape: (5, 2)
┌─────────┬────────┐
│ overall ┆ n      │
│ ---     ┆ ---    │
│ f64     ┆ u32    │
╞═════════╪════════╡
│ 1.0     ┆ 21204  │
│ 2.0     ┆ 10295  │
│ 3.0     ┆ 13762  │
│ 4.0     ┆ 22904  │
│ 5.0     ┆ 131835 │
└─────────┴────────┘
shape: (1, 2)
┌──────┬────────┐
│ year ┆ n      │
│ ---  ┆ ---    │
│ i64  ┆ u32    │
╞══════╪════════╡
│ 2018 ┆ 200000 │
└──────┴────────┘


In [7]:
user_counts = lf.group_by("reviewerID").agg(pl.len().alias("n_reviews"))

print(
    user_counts.select([
        pl.len().alias("total_users"),
        (pl.col("n_reviews") == 1).mean().alias("pct_one_review"),
        (pl.col("n_reviews") >= 2).mean().alias("pct_two_plus"),
        pl.col("n_reviews").median().alias("median_reviews_per_user"),
        pl.col("n_reviews").quantile(0.9).alias("p90_reviews_per_user"),
    ]).collect()
)


shape: (1, 5)
┌─────────────┬────────────────┬──────────────┬─────────────────────────┬──────────────────────┐
│ total_users ┆ pct_one_review ┆ pct_two_plus ┆ median_reviews_per_user ┆ p90_reviews_per_user │
│ ---         ┆ ---            ┆ ---          ┆ ---                     ┆ ---                  │
│ u32         ┆ f64            ┆ f64          ┆ f64                     ┆ f64                  │
╞═════════════╪════════════════╪══════════════╪═════════════════════════╪══════════════════════╡
│ 125233      ┆ 0.724442       ┆ 0.275558     ┆ 1.0                     ┆ 3.0                  │
└─────────────┴────────────────┴──────────────┴─────────────────────────┴──────────────────────┘


In [8]:
# First review date per user
first = (
    lf.group_by("reviewerID")
      .agg(pl.col("review_dt").min().alias("first_review_dt"))
)

NEW_DAYS = 90

lf2 = (
    lf.join(first, on="reviewerID", how="inner")
      .with_columns(
          (pl.col("review_dt") - pl.col("first_review_dt")).dt.total_days().alias("account_age_days")
      )
)

new_user_reviews = lf2.filter(pl.col("account_age_days") <= NEW_DAYS)

# Early vs late windows (within first 90 days)
early_end = 30
late_start, late_end = 60, 90

user_change = (
    new_user_reviews
      .with_columns(
          pl.when(pl.col("account_age_days") <= early_end).then(pl.lit("early"))
            .when((pl.col("account_age_days") >= late_start) & (pl.col("account_age_days") <= late_end)).then(pl.lit("late"))
            .otherwise(None)
            .alias("period")
      )
      .filter(pl.col("period").is_not_null())
      .group_by(["reviewerID", "period"])
      .agg(pl.mean("overall").alias("avg_rating"))
      .collect()  # pivot happens on eager DF
      .pivot(index="reviewerID", columns="period", values="avg_rating")
      .with_columns((pl.col("late") - pl.col("early")).alias("delta_rating"))
)

print(
    user_change.select([
        pl.len().alias("n_users_with_both_windows"),
        pl.col("delta_rating").mean().alias("mean_delta"),
        pl.col("delta_rating").median().alias("median_delta"),
        pl.col("delta_rating").quantile(0.25).alias("q25"),
        pl.col("delta_rating").quantile(0.75).alias("q75"),
    ])
)


shape: (1, 5)
┌───────────────────────────┬────────────┬──────────────┬───────────┬─────┐
│ n_users_with_both_windows ┆ mean_delta ┆ median_delta ┆ q25       ┆ q75 │
│ ---                       ┆ ---        ┆ ---          ┆ ---       ┆ --- │
│ u32                       ┆ f64        ┆ f64          ┆ f64       ┆ f64 │
╞═══════════════════════════╪════════════╪══════════════╪═══════════╪═════╡
│ 125233                    ┆ 0.017225   ┆ 0.0          ┆ -0.166667 ┆ 0.5 │
└───────────────────────────┴────────────┴──────────────┴───────────┴─────┘


/var/folders/jc/m5_dzhb94fsdm0nt78jxzpx00000gn/T/ipykernel_11573/1922272560.py:34: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  .pivot(index="reviewerID", columns="period", values="avg_rating")
